# Análisis Exploratorio de Supermercado Santa Isabel

<img src="https://play-lh.googleusercontent.com/gROtNrj8vGhnMKf0QyPgmMCbQU4R8wFZ329H-aO-M7OLTOzzW2zHM1wEgqdgF7Bo6es" alt="Logo_Santa_Isabel" width="200"/>

## Resumen del proyecto 
En este proyecto, investigo el dataset de productos del supermecado [santaisabel.cl](https://www.santaisabel.cl/). Con el uso de las bibliotecas Python Scipy, NumPy, Pandas, Matplotlib y Seaborn.

### Información del conjunto de datos

**Datos de santaisabel** : Contiene datos de nombre, precios, marcas y categoría ofrecidos en su pagina web de 4470 productos

| Variable     	| Definición                                           	| Ejemplo                                                            	|   	|   	|
|--------------	|------------------------------------------------------	|--------------------------------------------------------------------	|---	|---	|
| title        	| Es el titulo del producto                            	| Pasta dental triple 3 un. 90 g                                     	|   	|   	|
| brand        	| Es la marca del producto                             	| Pepsodent                                                          	|   	|   	|
| normal_price 	| Es el precio normal                                  	| 2.790                                                              	|   	|   	|
| low_price    	| Es el precio con oferta                              	| 2.590                                                              	|   	|   	|
| high_price   	| Es el precio antes de la oferta                      	| 3.190                                                              	|   	|   	|
| oferta       	| Si el producto tiene una oferta                      	| 1                                                                  	|   	|   	|
| categoria1   	| Categoría de los productos                            	| supermercados                                                      	|   	|   	|
| categoria2   	| Subcategoria de los productos                        	| belleza-y-cuidado-personal                                         	|   	|   	|
| datetime     	| Fecha en que se recoletaron los datos                	|                                                         2022-07-16 	|   	|   	|
| Atributos    	| Es el peso del producto                              	| 500 g                                                              	|   	|   	|
| link         	| Enlace hacia el producto                             	| https://www.santaisabel.cl/queso-mantecoso-granulado-c-y-co-250g/p 	|   	|   	|
| sin_stock    	| Si el producto esta disponible                       	| o                                                                  	|   	|   	|
| page         	| Paginas donde se encuentra el producto en la website 	| 3                                                                  	|   	|   	|

## Preguntas iniciales e investigación de datos 
-  ¿ cuál es la distribución de precios de todos los productos ?
-  ¿ cuál es la cantidad de productos con ofertas con respecto a los que no lo tienen ? 
-  ¿ cuál es el producto más caro ?
-  ¿ cuál es la distribución del precio de los productos con respecto a su categoría y su sub categoría?
-  ¿ cuál es la marca mas frecuente en santaisabel. Que posee mayor cantidad de productos ? 
-  ¿ cuales son los productos que tienen un mayor descuento y cuales son sus porcentaje ?
-  ¿ cuales son las palabras más frecuentes de los productos ?
    
Cargaremos las bibliotecas de Python necesarias para nuestro análisis y leeremos nuestro dataset:

In [ ]:
pip install plotly

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS
import re


In [ ]:
#concateno varios dataset en un solo dataset
result = pd.read_csv('../input/santa-isabel-chile/result.csv')
result.head()

In [ ]:
result.info()

## Limpieza de datos
No usaremos la columna 'link' para esta investigación inicial. 

In [ ]:
#result.drop('link',axis=1,inplace=True)

Revisamos si hay valores nulos 

In [ ]:
result.isnull().sum()

In [ ]:
result.empty

In [ ]:
# Check if all values are zero in column 'Budget'
for column in list(result):
    if (result[column] == 0).any():
        print(f"All values in the column '{column}' are Zero")
    else:
        print(f"All values in the column '{column}' are not Zero")


In [ ]:
zero_price = result.loc[result["normal_price"]==0, "normal_price"].value_counts().sort_index()
zero_price

In [ ]:
f'price with zero values {round((zero_price/len(result) )* 100,2)} %'

In [ ]:
# cambiamos el tipo de columna de fecha de un object a un timetime para poder trabajar con fechas 
result['datetime'] = pd.to_datetime(result['datetime'])

creación de una función para limpiar los signos de las monedas y obtener solo el valor sin comas y le cambiamos su tipo a flotante

In [ ]:
def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return(x.replace('$', '').replace('.', '').replace('Normal:','').replace(',',''))
    return(x)

result['low_price'] = result['low_price'].apply(clean_currency).astype('float')
result['high_price'] = result['high_price'].apply(clean_currency).astype('float')
result['normal_price'] = result['normal_price'].apply(clean_currency).astype('float')
#df['low_price'] = df['low_price'].str.replace(',', '').str.replace('$', '')
#df['low_price'].astype(np.float64)
#df['low_price'] = pd.to_numeric(df['low_price'],errors = 'coerce')
#df['low_price'].astype('float')
result.info()

cambiamos el valor 'Producto sin stock' a 0 y si hay stock a 1
para oferta se hace igual cuando hay oferta se cambia a 1 y cuando no se cambia a cero 

In [ ]:
result['sin_stock'] = result['sin_stock'].apply(lambda x:0 if x=='Producto sin stock' else 1)
result['oferta'] = result['oferta'].apply(lambda x:1 if x=='Oferta' else 0)

### valores duplicados
Revisamos si hay valores duplicados en caso de conseguirlos se eliminan 

In [ ]:
result.duplicated().sum()

In [ ]:
result[result.duplicated()==True]

In [ ]:
result.drop_duplicates(keep=False,inplace=True)

## Ingeniería de Características

Preparar datos de entrada relevantes que sean compatibles con los requisitos del algoritmo

In [ ]:
#obtener la diferencia del precio mas alto con respecto al descuento y cuardalo en la variable ahorro
result['ahorro'] = result['high_price'] - result['low_price']

In [ ]:
#obtener el ahorro en porcentajes
result['ahorro_percent'] = (result['ahorro'] / 
                      result['ahorro'].sum()) * 100

#que solo muestre los primeros 3 decimales
result['ahorro_percent'] = round(result['ahorro_percent'],3)
result.head(3)


convertir todas las unidades a kilogramos  

In [ ]:

result['Atributos'].unique()


In [ ]:
def to_kilos(arg):
    """convierto todas las unidades a kilos"""
    valor =  re.findall(r'-?\d+\.?\d*', arg)
    unidad = re.findall(r'[a-z]+$', arg)
    unidad =  ''.join(unidad)
    valor = ''.join(valor)
    valor = pd.to_numeric(valor)
    if unidad == 'g':
        return valor /1000.0
    else: 
        return valor

    
result['kilo'] = result['Atributos'].apply(to_kilos)
result['kilo']

## Analisis de Dato

### Obtener los 10 productos que ofrecen mayor descuentos

In [ ]:
result[result['ahorro']>0].sort_values(by='ahorro',ascending = False)[['title','low_price','normal_price','high_price','ahorro','ahorro_percent']].head(10)

La cerveza son los productos que tiene una mayor variabilidad de precio. Cuando se encuentra en oferta se puede ahorrar 9500 pesos o 14 %
seria interesante como varían las ofertas y precio en el tiempo

In [ ]:
# Make a function to get the composition of the variables per number of products
def Groupby_OneCol_comp_plot(df, col, plt_style = 'seaborn-ticks', color_palette = "coolwarm"):
    '''
    Group by col1, sort by size , return and plot the dataframe with a bar and pie plot
    link -> https://jkarakas.github.io/Exploratory-Analysis-of-the-Titanic-Dataset/Titanic_Dataset_Exploratory_Analysis._With_Code.html
    '''
    gr=pd.DataFrame()
    gr['{} No'.format(col)] = df.groupby(col).size()
    gr['{} Ratio'.format(col)] = np.round(gr['{} No'.format(col)].divide(gr['{} No'.format(col)].sum())*100,0)
    
    print ('Total No. of {}:{}'.format(col,gr['{} No'.format(col)].sum()))
    
    plt.style.use(plt_style)
    sns.set_palette(sns.color_palette(color_palette))
    
    fig=plt.figure()
    plt.axis('off')

    fig.add_subplot(121)
    
    ax=gr['{} No'.format(col)].plot(kind='bar', title='{} Counts'.format(col), figsize=(16,8), color=sns.color_palette())
    _ = plt.setp(ax.get_xticklabels(), rotation=0)
    for p in ax.patches: ax.annotate(np.round(p.get_height(),decimals=2),
                                     (p.get_x()+p.get_width()/2., p.get_height()),
                                     ha='center', va='center', xytext=(0, 10), textcoords='offset points')
    ax.get_yaxis().set_ticks([])
    plt.xlabel('')

    fig.add_subplot(122)
    plt.axis('off')
    gr.loc[:,'{} Ratio'.format(col)].plot(kind= 'pie',
                                     autopct='%1.1f%%',shadow=False,
                                     title='{} Ratio'.format(col), legend=False, labels=None);
    sns.despine(top=True, right=True, left=True, bottom=False);

### Análisis de la Categoría.
iniciamos visualizando la cantidad de productos por la categoría a la que pertenece

In [ ]:
Groupby_OneCol_comp_plot(result, 'categoria1')

Vemos que la mayor cantidad de productos se encuentra en la categoría de despensa (2485 de 4470 - 56%) <br>
seguido de la categoría supermercado con (1092 de 4470 - 24%) <br>
y la categoría con menor productos es carnicería con (83 de 4470 - 20%)


In [ ]:
def plot(table, legloc='upper right',
                                    plt_style = 'seaborn-ticks',
                                    color_palette="dark",sorter=None, stacked=False,
                                    kind = 'bar', percentage = True,
                               custom_title=None, minimal=True, figsize=(19,10), width=0.7 ):     
    grouped = table
    
    #Tranform to percentages
    if percentage == True:
        grouped = np.round(grouped.divide(grouped['Total'],axis=0)*100,0)
    try:   
        del grouped['Total']
    except:
        pass
    
    # rearrange the columns
    if sorter:
        grouped = grouped[sorter]

    plt.style.use(plt_style)
    sns.set_palette(sns.color_palette(color_palette))
    ax = grouped.plot(kind=kind,stacked=stacked, figsize=figsize, width=width)
    _ = plt.setp(ax.get_xticklabels(), rotation=0)  # Rotate labels
    plt.legend(loc=legloc) # plot the legend normally
    
    #annotate the bars
    if percentage == True:
      for p in ax.patches:
            ax.annotate('{}%'.format(int(np.round(p.get_height(),decimals=2))),
                                         (p.get_x()+p.get_width()/2.,
                                          p.get_height()), ha='center', va='center',
                                        xytext=(0, 10), textcoords='offset points')
    else:
      for p in ax.patches:
            ax.annotate(np.round(p.get_height(),decimals=2),
                                         (p.get_x()+p.get_width()/2.,
                                          p.get_height()), ha='center', va='center',
                                        xytext=(0, 10), textcoords='offset points')
    if minimal == True:
        ax.get_yaxis().set_ticks([])
        plt.xlabel('')
        sns.despine(top=True, right=True, left=True, bottom=False);
    else:
        pass     
    # set custom title    
    plt.title(custom_title)
    
def Groupby_TwoCol_Plot(df, col1, col2, legloc='upper right',
                                    plt_style = 'ggplot',
                                    color_palette="dark",sorter=None, stacked=False,
                                    kind = 'bar', percentage = True,
                               custom_title=None, minimal=True, figsize=(14,6), width=0.6):   
    
    #Group by Placement and Representative and unstack by Placement
    grouped = df.groupby([col2,col1]).size().unstack(col2)
    
    #Make a totals column sort and delete after
    grouped['Total'] = grouped.sum(axis=1)
    #grouped = grouped.sort_values('Total', ascending = False)
   
    plot(grouped, legloc=legloc,
                                    plt_style = plt_style,
                                    color_palette=color_palette,sorter=sorter, stacked=stacked,
                                    kind = kind , percentage = percentage,
                               custom_title=custom_title, minimal=minimal, figsize=figsize, width=width)

In [ ]:
Groupby_TwoCol_Plot(result,'categoria1', 'oferta', color_palette=('darkred','steelblue'),
                    plt_style = 'seaborn-ticks', custom_title='Proporción de oferta por categoría')

Santaisabel ofrece descuento en productos de supermercado. luego en productos de carnicería. esto posiblemente se debe a que tienden a dañarse rápidamente

### Distribución de Precios por Categoría

In [ ]:
#plt.figure(figsize=(15,8))
#sns.boxplot(data=result,x='categoria1',y='normal_price')
fig = px.box(result, x="categoria1", y="normal_price", 
             notched=True,template="seaborn",
             color_discrete_sequence=px.colors.qualitative.Dark2,
            color='categoria1')
fig.show()

In [ ]:
#plt.figure(figsize=(15,8))
#sns.violinplot(data=result,x='categoria1',y='normal_price')
px.violin(result, x="categoria1", y="normal_price",template="seaborn",
             color_discrete_sequence=px.colors.qualitative.Dark2,
            color='categoria1')
 


In [ ]:
result.groupby(['categoria1'])['normal_price'].agg([np.mean,np.sum,max])

vemos como se distribuyen los precios de acuerdo a la categorías. en donde carnicería tiene una mediana mas alta que otras categoría ademas tiene una mayor dispersión, es decir una mayor variedad de precios. mientras que frutas y verduras es la categoría con una menor variabilidad. lo cual quiere decir que los precios de frutas y verduras son muy similares. todos los productos tiene su mayor densidad de valor por debajo de los 10 mil pesos. lo cual significa que es muy extraño conseguir productos con un valor por encima de 10 mil pesos
<br>
**nota**:todas las categorías tienen valores atípicos

### Analisis de SubCategorias 

#### Distribución del Precio y la Sub Categoría 

De las subcategorias. La que tiene una mayor dispersión es la carne de vacuno seguido de la pescaderia en cambio en postres, instantáneos-salsa y frutas tienen poca variabilidad
y su valor tiende a cero y el que tiene mas valores atípico son los productos lácteos

In [ ]:
#plt.figure(figsize=(20,8))
#ax = sns.boxplot(data=result,x='categoria2',y='normal_price')
#ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
px.violin(result, x="categoria2", y="normal_price",template="seaborn",
             color_discrete_sequence=px.colors.qualitative.Dark2,
            color='categoria1')


In [ ]:
#Groupby_TwoCol_Plot(result,'categoria2', 'oferta', color_palette=('darkred','steelblue'), plt_style = 'seaborn-ticks', custom_title='Proporción de oferta por subcategoría')

### Tabla de precios promedio, máximo y la suma del precio por Sub Categoría 

In [ ]:
tabla = result.groupby(['categoria2'])['normal_price'].agg([np.mean,np.sum,max]).reset_index()
#Tabla ordenado del valor máximo al mínimo Top 5
#La sub categoría  de repostería tiene el producto mas caro 
tabla.sort_values(by='max',ascending=False).head()

In [ ]:
# para saber cual es ese producto
# crema de leche de 1 litro con un precio de cuarenta y cuatro mil pesos
result.sort_values(by='normal_price',ascending=False)[:1]

In [ ]:
#tabla ordenado de la suma de valores (precio)
#sumando todos los precios la sub categoría lácteos y leche acumulan mayor valor monetario
tabla.sort_values(by='sum',ascending=False).head()

In [ ]:
#tabla ordenado del valor promedio al minimo Top 5
#Al obtener la el promedio la sub categoría de vacuno se lleva el premio 
tabla.sort_values(by='mean',ascending=False).head()

### Distribución de los precios 

In [ ]:
prices = result[['normal_price','low_price','high_price']]

In [ ]:
plt.figure(figsize=(15,8))

low_price = result[result['low_price']>0]['low_price'].dropna()
normal_price = result[result['normal_price']>0].dropna()
high_price = result[result['high_price']>0]['high_price'].dropna()
#sns.distplot(low_price)
#sns.distplot(normal_price)
#sns.distplot(high_price)

#px.histogram(result, x="normal_price", y="oferta", marginal="rug", hover_data=result.columns)
import plotly.figure_factory as ff
fig = ff.create_distplot(
    [normal_price['normal_price'].tolist()],
    group_labels=['normal_price'],
    show_hist=False,
).add_traces(
    px.histogram(normal_price, x="normal_price")
    .update_traces(yaxis="y", name="histogram")
    .data
).update_layout(yaxis3={"overlaying": "y", "side": "right"}, showlegend=False)

fig

In [ ]:
ax=plt.figure(figsize=(15,8))
plt.suptitle('Passenger Age Distribution')
ax.add_subplot(121)
price = result[result['normal_price']>0]['normal_price']
sns.distplot(price,bins=11)
ax.add_subplot(122)
sns.boxplot(price);

Una pregunta interesante es donde esta acumulada la mayor densidad del valor en pesos de todos los productos ?
En la grafica se nos muestra que de 0 a 10 mil pesos chilenos es donde se acumula el mayor valor. Esto implica que la mayoría 
de los productos que se encuentran disponibles están en un rango de 10 mil pesos.

### Analisis de Ofertas

In [ ]:
oferta_count = result.groupby('oferta')['oferta'].count()
oferta_count

In [ ]:
plt.figure(figsize=(4,5))
plt.bar(oferta_count.index, oferta_count.values)
plt.title('Grouped by ofertas')
plt.xticks([0,1],['Not oferta', 'oferta'])
for i, value in enumerate(oferta_count.values):
    plt.text(i, value-120, str(value), fontsize=12, color='white',
             horizontalalignment='center', verticalalignment='center')
plt.show()

3611 productos no tiene oferta mientas que 857 si tiene ofertas 

### Análisis de Stock

In [ ]:
no_stock_count = result.groupby('sin_stock')['sin_stock'].count()
no_stock_count

In [ ]:
plt.figure(figsize=(4,5))
plt.bar(no_stock_count.index, no_stock_count.values)
plt.title('Grouped by stock')
plt.xticks([0,1],['Not stock', 'stock'])
for i, value in enumerate(oferta_count.values):
    plt.text(i, value-120, str(value), fontsize=12, color='white',
             horizontalalignment='center', verticalalignment='center')
plt.show()

Solo 207 productos no están disponibles en el dataset

### Análisis de Marcas

 - ¿ cuales son las marcas con mayor cantidad de productos ?

In [ ]:
no_stock_count = result.groupby('brand')['brand'].count()
no_stock_count.sort_values(ascending=False)[:10]

    - ¿ cuales de esas marcas tienen los precios mas caros y el mayor promedio y cuanto suman su valor en pesos sus productos ?

In [ ]:
result.groupby(['brand']).agg({'normal_price':sum}).sort_values(by='normal_price',ascending=False).head(10)


Couisine & Co es el que suma mas valor monetario con un total de un millón ciento setenta y tres mil pesos con ciento setenta y cinco pesos

In [ ]:
result.groupby(['brand']).agg({'normal_price':np.mean}).sort_values(by='normal_price',ascending=False).head(10)

El que tiene un promedio mas elevado es la marca chivas Regal con treinta y dos mil pesos

In [ ]:
result.groupby(['brand']).agg({'normal_price':max}).sort_values(by='normal_price',ascending=False).head(10)

Coisine & Co es la marca que tiene el producto mas caro

### Distribución de los precios por categorías

In [ ]:
result.hist(column="normal_price",by="categoria1",sharey=True,figsize=(15,8))
plt.suptitle('Distribución de los precios por categorías');

In [ ]:
result.hist(column="normal_price",by="categoria2",sharey=True,figsize=(20,12))
plt.suptitle('Distribución de los precios por sub categorías');

## Palabras mas Comunes en los productos 

In [ ]:
comment_words = ''
stopwords = set(STOPWORDS)
 
    
# iterate through the csv file
for val in result['title']:
     
    # typecaste each val to string
    val = str(val)
 
    # split the value
    tokens = val.split()
     
    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
     
    comment_words += " ".join(tokens)+" "
 
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(comment_words)
 
# plot the WordCloud image                      
plt.figure(figsize = (8, 12), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
 
plt.show()

In [ ]:
#result['title'].value_counts(dropna=False).head(15)

## Respuesta a las Preguntas iniciales
-  ¿ cual es la distribución de precios de todos los productos ?
    - R) los precios siguen una distribución normal sesgada a la derecha. lo cual muestra que la mayor densidad de los precios Se encuentra entre 0 a 5.000 pesos
-  ¿ cual es la cantidad de productos con ofertas con respecto a los que no lo tienen ?
    - R) hay un total de 857 productos con ofertas 
-  ¿ cual es el producto mas caro ?
    - R) crema de leche de 1 litro con un precio de cuarenta y cuatro mil pesos
-  ¿ cual es la distribución del precio de los productos con respecto a su categoría y su sub categoría?
    - R) Ver grafico
-  ¿ cual es la marca mas frecuente en santaisabel. Que posee mayor cantidad de productos ?
    - R) Soprole y Colun son las marcas mas frecuentes 
-  ¿ cuales son los productos que tienen un mayor descuento y cuales son sus porcentaje ?
    - R) Pack 24 un. Cerveza lager 5.2° 330 cc con un descuento del 14% y un ahorro de 9500.0 pesos 
    el segundo producto es 'Caja 24 un. cerveza lager botella 330 cc c/u' con un descuento del 14% y un ahorro de 9280.0 pesos
-  ¿ cuales son las palabras mas frecuentes de los productos ?
    - R) Ver grafico de arriba 

In [ ]:
#guardar el dataset
result.to_csv('result.csv',index=False,encoding='utf-8', header='true')